<a href="https://colab.research.google.com/github/Gorik27/MD_MC_course/blob/main/LAMMPS_Task_1_NVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of LAMMPS



In [1]:
!apt install cmake build-essential git ccache

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.12).
The following additional packages will be installed:
  libhiredis0.14
Suggested packages:
  distcc | icecc
The following NEW packages will be installed:
  ccache libhiredis0.14
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 528 kB of archives.
After this operation, 1,469 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhiredis0.14 amd64 0.14.1-2 [32.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ccache amd64 4.5.1-1 [495 kB]
Fetched 528 kB in 1s (597 kB/s)
Selecting previously unselected package libhiredis0.14:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .

In [2]:
!rm -rf lammps
!git clone -b stable --depth=1 https://github.com/lammps/lammps.git lammps

Cloning into 'lammps'...
remote: Enumerating objects: 13988, done.
remote: Counting objects: 100% (13988/13988), done.
remote: Compressing objects: 100% (10097/10097), done.
remote: Total 13988 (delta 4771), reused 7738 (delta 3659), pack-reused 0 (from 0)
Receiving objects: 100% (13988/13988), 128.14 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (4771/4771), done.
Updating files: 100% (13334/13334), done.


In [3]:
!rm -rf build
!mkdir build
!cd build; cmake -C ../lammps/cmake/presets/basic.cmake -D CMAKE_INSTALL_PREFIX=/usr -D CMAKE_CXX_COMPILER_LAUNCHER=ccache -D BUILD_SHARED_LIBS=on -D LAMMPS_EXCEPTIONS=on -D PKG_PYTHON=on ../lammps/cmake
!cd build; make

loading initial cache file ../lammps/cmake/presets/basic.cmake
CMake Deprecation Warning at CMakeLists.txt:18 (cmake_policy):
  The OLD behavior for policy CMP0109 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explains that the OLD behaviors of all
  policies are deprecated and that a policy should be set to OLD only under
  specific short-term circumstances.  Projects should be ported to the NEW
  behavior and not rely on setting a policy to OLD.


-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Appending /usr/local/cuda/lib64/stubs to CMAKE_LIBRARY_PATH: /usr/local/cuda/lib64/stubs
-- Running check for auto-generated files from make-based build system
-- Found MPI_CXX: /usr/lib/x86_64-li

In [4]:
!cd build; make install

[  0%] Built target citeme.h
[  0%] Built target library.h
[  0%] Built target lmppython.h
[  0%] Built target lmptype.h
[  0%] Built target memory.h
[  0%] Built target modify.h
[  1%] Built target neighbor.h
[  1%] Built target neigh_list.h
[  1%] Built target output.h
[  1%] Built target pair.h
[  1%] Built target platform.h
[  1%] Built target force.h
[  2%] Built target group.h
[  2%] Built target improper.h
[  2%] Built target input.h
[  2%] Built target utils.h
[  3%] Built target variable.h
[  3%] Built target fmt_core.h
[  3%] Built target fmt_format.h
[  3%] Built target comm.h
[  3%] Built target command.h
[  3%] Built target compute.h
[  4%] Built target dihedral.h
-- Git Directory: /content/lammps/.git
-- Generating lmpgitversion.h...
[  4%] Built target gitversion
[  4%] Built target angle.h
[  4%] Built target pointers.h
[  5%] Built target region.h
[  5%] Built target timer.h
[  5%] Built target universe.h
[  5%] Built target update.h
[  5%] Built target info.h
[  6%] B

Writing LAMMPS input file
* LAMMPS manual https://www.afs.enea.it/software/lammps/doc17/html/Section_commands.html

In [5]:
%%writefile in.create

######################################
# INITIALIZATION
clear
units metal
dimension 3
boundary p p p
atom_style atomic
atom_modify map array

######################################
# ATOM DEFINITION
lattice fcc 4 orient x 1 1 0 orient y -1 1 0 orient z 0 0 1
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
create_atoms 1 box
replicate 10 10 15

######################################
# DEFINE INTERATOMIC POTENTIAL
pair_style eam/alloy
pair_coeff * * Al99.eam.alloy Al
neighbor 2.0 bin
neigh_modify delay 10 check yes

######################################
# DEFINE COMPUTES
compute eng all pe/atom
compute eatoms all reduce sum c_eng

#####################################################
# MINIMIZATION
reset_timestep 0
fix 1 all box/relax iso 0.0 vmax 0.001
thermo 10
thermo_style custom step pe lx ly lz press c_eatoms
min_style cg
minimize 1e-25 1e-25 5000 10000

variable natoms equal "count(all)"
variable teng equal "c_eatoms"
variable length equal "lx"
variable ecoh equal "v_teng/v_natoms"

write_data crystal.dat

print "Total energy (eV) = ${teng};"
print "Number of atoms = ${natoms};"
print "Lattice constant (Angstoms) = ${length};"
print "Cohesive energy (eV) = ${ecoh};"

print "All done!"

Writing in.create


Running LAMMPS

In [14]:
! lmp -in in.create

LAMMPS (29 Aug 2024 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 5.6568542 5.6568542 4
Created orthogonal box = (0 0 0) to (5.6568542 5.6568542 4)
  1 by 1 by 1 MPI processor grid
Created 8 atoms
  using lattice units in orthogonal box = (0 0 0) to (5.6568542 5.6568542 4)
  create_atoms CPU = 0.000 seconds
Replication is creating a 10x10x15 = 1500 times larger system...
  orthogonal box = (0 0 0) to (56.568542 56.568542 60)
  1 by 1 by 1 MPI processor grid
  12000 atoms
  replicate CPU = 0.002 seconds
Switching to 'neigh_modify every 1 delay 0 check yes' setting during minimization
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.28721
  ghost 

Visualisation using package OVITO

*   Scripting manual https://www.ovito.org/manual/python/index.html
*   Standalone program (there is a free version) https://www.ovito.org/



In [8]:
# installation of OVITO package
!pip install -U ovito

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 550.5/550.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.6/160.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 52.3 MB/s eta 0:00:00


In [3]:
# support for third-party widjets in colab
from google.colab import output
output.enable_custom_widget_manager()

In [7]:
# visualisation
from ovito import scene
from ovito.io import import_file
from ovito.vis import Viewport, OpenGLRenderer
from ipywidgets import Layout

for pip in scene.pipelines:
  pip.remove_from_scene()
pipeline = import_file("crystal.dat")
pipeline.add_to_scene()

vp = Viewport(type=Viewport.Type.Perspective, camera_dir=(0.5, 1.0, -0.4))
widget = vp.create_jupyter_widget(layout=Layout(width='80%', height='100%'),
                                  picking=True)
display(widget)

JupyterViewportWidget(layout=Layout(height='100%', width='80%'), picking=True)

**Task 1: NVE ensemble**


---



1.   Generate model of crystal, using script "in.create" and visualize it using OVITO
2.   Write a script for MD simulation. It should contain the commands: ```read_data``` (to load the structure calculated by "in.create"), ```velocity``` (to initialize initial velocities), ```fix``` (style ```nve```; this command sets the integrator), ```run``` and ```dump``` (to save snapshots of atomic positions and velosities for further analysis and visualization). Description and syntax of these commands can be found in LAMMPS user manual: https://www.afs.enea.it/software/lammps/doc17/html/Section_commands.html#commands-listed-by-category (documentation can also be found on the LAMMPS website, but it is unaccessible without vpn https://docs.lammps.org)
3. Set initial velosities to be uniformly distributed and check that they relax to the Maxwell distribution after a simulation. (velocities can be loaded from dump files using OVITO
4. Calculate temperature. Why did the initial temperature drop specifically by a factor of two?
5. Check the conservation of energy (potential + kinetic). Find the maximum time step  that keeps energy conservation. Compare it with critical time step for harmonic oscillator.

Commands to write dump:
```
# LAMMPS script
shell mkdir dumps #make a folder "dumps"
dump 1 all cfg 100 dumps/dump_*.cfg mass type xs ys zs vx vy vz id # write dump file each 100 steps
dump_modify 1 element Al
```

Commands to write thermo data to file "thermo.txt":
```
# LAMMPS script
variable outfile index "thermo.txt" # initialise variable
print "# thermo output" file ${outfile} screen no # write header to file
fix out all print 10 "$(step) $(temp) $(pe)" append ${outfile} screen no # initialise fix that will print step; temperature; potential energy to file each 10 steps
```

Commands to load dump files ```("dumps/dump_*.cfg")``` in OVITO and visualize it in Jupiter:

In [8]:
from ovito import scene
for pipl in scene.pipelines:
  pipl.remove_from_scene()

from ipywidgets import interact, Layout, IntSlider
from glob import glob

dumps = glob('dumps/dump_*.cfg')
widget = vp.create_jupyter_widget(layout=Layout(width='80%', height='100%'),
                                  picking=True)
display(widget)
vp = Viewport(type=Viewport.Type.Perspective, camera_dir=(0.5, 1.0, -0.4))


def view(n):
  for pip in scene.pipelines:
    pip.remove_from_scene()
  pipeline = import_file(dumps[n])
  pipeline.add_to_scene()
  widget.refresh()

slider = IntSlider(min=0, max=len(dumps)-1, step=1, value=0)
interact(view, n=slider);

JupyterViewportWidget(layout=Layout(height='100%', width='80%'), picking=True)

TraitError: setting min > max

Commands to load particle  velocities:
```
# python
pipeline = import_file('dumps/dump_*.cfg')
for frame in range(pipeline.num_frames):
  data = pipeline.compute(frame) # access data of frames from 0 to pipeline.num_frames
  v3s = data.particles['Velocity'].array
```
List of other quantities:
```list(data.particles.keys())```

**P.S.**


---

Sometimes it is useful to use Python to run a series of LAMPPS simulations. LAMMPS can be invoked from Python in different ways, which are described in user's manual.  However, I prefer to call it as in the command line using the subprocess module:

In [13]:
from subprocess import Popen, PIPE
import time

nonverbose = True #not print every line
script = 'in.create'
variable1 = 1
variable2 = 1

task = f'lmp -in {script} -var variable1 {variable1} -var variable2 {variable2}'
exitflag = False
error_msg = ''

print('starting LAMMPS...')
print(task)
with Popen(task.split(), stdout=PIPE, bufsize=1, universal_newlines=True) as p:
  time.sleep(0.1)
  print('\n')
  for line in p.stdout:
    if "thermo output" in line:
      thermo = (line.replace('thermo output ', '')).replace('\n', '')
    elif "All done" in line:
      exitflag = True
    elif "ERROR" in line:
      error_msg = line.replace('ERROR: ', '')
    if not nonverbose:
      if '!' in line:
        print(line.replace('!', ''), end='')
      else:
        print(line, end='')

if not exitflag:
  raise ValueError(f'Error in LAMMPS: {error_msg}')


starting LAMMPS...
lmp -in in.create -var variable1 1 -var variable2 1




here we can put some input data to lammps script via keyword ```-var varname value```, where ```varname``` is the name of variable in lammps input script (initialized as ```variable varname index ...```), ```value``` is a sending value of this variable